In [10]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [2]:
os.makedirs(db_dir,exist_ok=True)

In [3]:
if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

In [4]:
browser=helium.start_chrome("https://www.strava.com/login")

In [5]:
helium.write('diego.restrepo@gmail.com',into='Your Email')

In [6]:
helium.write('opoqdh11',into='Password')

In [7]:
helium.click('Log In')

In [19]:
#Required to activate card-section below
browser.maximize_window()

In [20]:
time.sleep(10)
lt=[t for t in browser.find_elements_by_class_name('card-section') 
   if t.text.find('Latest Activity')>-1][0]

In [21]:
helium.click( lt.find_element_by_tag_name('a').text )

Web page with latest activity

General info

In [22]:
time.sleep(10)
dt=browser.find_element_by_class_name('details')

In [23]:
date=dt.find_element_by_tag_name('time').text
date

'1:49 PM on Sunday, May 24, 2020'

In [24]:
name=dt.find_element_by_tag_name('h1').text
name

'La Tuna'

In [25]:
kk=browser.find_elements_by_tag_name("ul")

In [26]:
q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

In [27]:
distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

In [28]:
distance,time,elevation

('8.20km', '1:03:18', '447m')

In [29]:
q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

In [30]:
power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )
power,energy

('144W', '548kJ')

Load tables

In [31]:
browser.find_element_by_id('show-hidden-efforts').click()

In [32]:
html=browser.page_source

In [33]:
ll=pd.read_html(html)

General info table

In [34]:
mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

In [35]:
mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

In [36]:
mtdt

{'Calories': {'Avg': '611', 'Max': '611'},
 'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:08:13'},
 'Speed': {'Avg': '7.8km/h', 'Max': '57.6km/h'},
 'date': '1:49 PM on Sunday, May 24, 2020',
 'distance': '8.20km',
 'elevation': '447m',
 'energy': '548kJ',
 'name': 'La Tuna',
 'power': '447m',
 'time': '1:03:18'}

Segments

In [37]:
sgmt=ll[1][['Name','Time','Speed','Power']]

In [38]:
sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

In [39]:
sgmt['Info']=''

In [40]:
sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [41]:
sgmt[:1]

,Name,Time,Speed,Power,Info
0,"Loma linda, detrás del antiguo principito 0.4...",2:26,11.4km/h,206W,"{'Elapsed Time': {'Max': '1:08:13', 'Avg': '1:..."


In [42]:
df=df.append(sgmt).reset_index(drop=True)

In [43]:
df.to_json(db_file)